# 9. lekce: Cvičení

## Kosatce
Data: [IRIS.csv](data/IRIS.csv)

![iris image](data/iris_image.png)
*Zdroj: https://www.analyticsvidhya.com/blog/2022/06/iris-flowers-classification-using-machine-learning/*

- Petal: Okvětní lístek
- Sepal: Kališní lístek

Datovou sadu s daty o kosetcích (Iris) představil britský statistik a biolog Ronald Fisher ve svém článku z roku 1936 o využití mnohonásobných měření v taxonomických problémech. Někdy se mu říká Andersonův kosatcový datový soubor, protože Edgar Anderson shromáždil data k měření morfologické variace květin kosatce tří souvisejících druhů. Datová sada obsahuje 50 vzorků od každého ze tří druhů kosatce (Iris Setosa, Iris virginica a Iris versicolor). Od každého vzorku byly změřeny čtyři vlastnosti: délka a šířka kališních lístků a korunních lístků, v centimetrech.

Tato datová sada se stala typickým testovacím případem pro mnoho statistických klasifikačních technik v strojovém učení, jako jsou například podpůrné vektorové stroje.

Využij algoritmy `KNeighborsClassifier` a `SVC` ke klasifikaci každého vzorku do jednoho ze tří druhů kosance.

Postup je stejný jako v lekci:

- Rozděl data na vstupní a výstupní proměnné.
- Využij `GridSearchCV` k nalezení nejlepšího parametru pro `KNeighborsClassifier` (počet sousedů) a `SVC` (strategie)
- Dále vyzkoušej, jestli by nebylo zajímavé převést data do více dimenzí. Porovnej výsledek lineárního kernelu (`kernel="linear"`) s kernelem (`kernel="rbf"`). Můžeš vyzkoušet obě strategie, tj. budeš mít ve slovníku `params` dva klíče a každý z klíčů bude mít seznam dvou možných hodnot. Jaká dvojice parametrů má nejlepší hodnotu metriky `accuracy`?

In [381]:
import pandas
import matplotlib.pyplot as plt

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

In [382]:
# Načtení dat
data1 = pandas.read_csv("data/IRIS.csv")

print(data1.shape)
data1.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


- Rozděl data na vstupní a výstupní proměnné.

In [383]:
# Rozdělení dat na vstupní proměnné a cílovou klasifikaci
X1 = data1.drop(columns=["species"])
y1 = data1["species"]

- Využij `GridSearchCV` k nalezení nejlepšího parametru pro `KNeighborsClassifier` (počet sousedů) a `SVC` (strategie)

(Rozdělení dat na trénovací a testovací sadu netřeba řešit, funkce si to zařídí sama)

In [384]:
# Vytvoření klasifikátoru KNN a slovníku s parametry k otestování
model_1 = KNeighborsClassifier()
params_1 = {"n_neighbors": range(1, 31, 2)}

# Otestování klasifikátorů se všemi kombinacemi parametrů
# CV: Cross-Validation
clf_1 = GridSearchCV(model_1, params_1, scoring="accuracy")
clf_1.fit(X1, y1)

# Zobrazení parametrů a skóre nejlepšího klasifikátoru
print(clf_1.best_params_)
print(f"Best accuracy: {round(clf_1.best_score_, 2)}")
# ... zaokrouhlení na dvě desetinná místa

{'n_neighbors': 7}
Best accuracy: 0.98


- Dále vyzkoušej, jestli by nebylo zajímavé převést data do více dimenzí. Porovnej výsledek lineárního kernelu (`kernel="linear"`) s kernelem (`kernel="rbf"`)

In [385]:
# Stejným způsobem otestujeme SVC klasifikátor, rovnou včetně kernelu rbf
# RBF: Radial Basis Function

# Vytvoření klasifikátoru SVC (Support Vector Classification) a slovníku s parametry k otestování
# OVO: One-vs-One
# OVR: One-vs-Rest
model_2 = SVC()
params_2 = {"kernel": ["linear", "rbf"], "decision_function_shape": ["ovo", "ovr"]}

# Otestování klasifikátorů se všemi kombinacemi parametrů
# CV: Cross-Validation
clf_2 = GridSearchCV(model_2, params_2, scoring="accuracy")
clf_2.fit(X1, y1)

# Zobrazení parametrů a skóre nejlepšího klasifikátoru
print(clf_2.best_params_)
print(f"Best accuracy: {round(clf_2.best_score_, 2)}")
# ... zaokrouhlení na dvě desetinná místa

{'decision_function_shape': 'ovo', 'kernel': 'linear'}
Best accuracy: 0.98


Pro zajímavost, všechny výsledky GridSearchCV se ukládají do atributu `cv_results_` ve formě slovníku:

In [386]:
clf_2.cv_results_

{'mean_fit_time': array([0.00722179, 0.00554171, 0.00431175, 0.00462217]),
 'std_fit_time': array([0.00169845, 0.00186346, 0.00046781, 0.00073599]),
 'mean_score_time': array([0.00789495, 0.00326052, 0.00284281, 0.00312009]),
 'std_score_time': array([0.00740706, 0.00058795, 0.00072037, 0.00021076]),
 'param_decision_function_shape': masked_array(data=['ovo', 'ovo', 'ovr', 'ovr'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'decision_function_shape': 'ovo', 'kernel': 'linear'},
  {'decision_function_shape': 'ovo', 'kernel': 'rbf'},
  {'decision_function_shape': 'ovr', 'kernel': 'linear'},
  {'decision_function_shape': 'ovr', 'kernel': 'rbf'}],
 'split0_test_score': array([0.96666667, 0.96666667, 0.96666667, 0.96666667]),
 'split1_test_score': array

## Bonus: Poruchy

Stáhni si data o poruchách ze souboru [predictive_maintenance.csv](data/predictive_maintenance.csv). Význam sloupců je následující:

- UID: jedinečný identifikátor v rozsahu 1 až 10000,
- ID produktu: skládá se z písmene L, M nebo H pro nízkou (50 % všech výrobků), střední (30 %) a vysokou (20 %) variantu kvality výrobku a sériového čísla specifického pro danou variantu,
- teplota vzduchu (K),
- teplota procesu (K),
- otáčky (ot/min),
- točivý moment (Nm),
- opotřebení nástroje (min),
- označení "selhání stroje" (pokud k němu došlo).

Tvým úkolem je vytvořit model, který bude predikovat poruchu stroje.

Proveď následující postup:

- Vyřaď z datasetu sloupce (jsou dva), které nemají pro analýzu význam.
- Podívej se, kolik typů poruch bylo objeveno. Vyřaď nejméně častou poruchu.
- Rozděl data na vstupní proměnné a výstupní proměnnou.
- Vyzkoušej algoritmy Support Vector Machine a K Nearest Neighbors ke klasifikaci poruchy stroje. Porovnej, který algoritmus dosáhl lepších výsledků.

In [387]:
# Načtení dat
data2 = pandas.read_csv("data/predictive_maintenance.csv")

print(data2.shape)
data2.head()

(10000, 10)


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


- Vyřaď z datasetu sloupce (jsou dva), které nemají pro analýzu význam.

In [388]:
data2 = data2.drop(columns=["UDI", "Product ID"])

- Podívej se, kolik typů poruch bylo objeveno. Vyřaď nejméně častou poruchu.

In [389]:
# Seskupení podle typu poruchy a jejich četnost
failures = data2.groupby("Failure Type").size()
display(failures)

# Nalezení nejméně časté poruchy a získání jejího názvu ve formě stringu
least_frequent = failures[failures == failures.min()]
least_frequent_name = least_frequent.index[0]
print(least_frequent)
print(least_frequent_name)

# Vyřazení nejméně časté poruchy z datasetu a ověření, že zmizela
data2 = data2[data2["Failure Type"] != least_frequent_name]

print(data2["Failure Type"].unique())
print(data2.shape)

Failure Type
Heat Dissipation Failure     112
No Failure                  9652
Overstrain Failure            78
Power Failure                 95
Random Failures               18
Tool Wear Failure             45
dtype: int64

Failure Type
Random Failures    18
dtype: int64
Random Failures
['No Failure' 'Power Failure' 'Tool Wear Failure' 'Overstrain Failure'
 'Heat Dissipation Failure']
(9982, 8)


- Rozděl data na vstupní proměnné a výstupní proměnnou.

In [390]:
# Rozdělení dat na vstupní proměnné a cílovou klasifikaci
X2 = data2.drop(columns=["Failure Type"])
y2 = data2["Failure Type"]

# Převod kategorických dat na dummies
# (vracíme se k pandas, jelikož OneHotEncoder by převedl všechny sloupce včetně číselných, ccož nechceme)
X2 = pandas.get_dummies(X2, columns=["Type"])
X2.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,False,False,True
1,298.2,308.7,1408,46.3,3,0,False,True,False
2,298.1,308.5,1498,49.4,5,0,False,True,False
3,298.2,308.6,1433,39.5,7,0,False,True,False
4,298.2,308.7,1408,40.0,9,0,False,True,False


- Vyzkoušej algoritmy Support Vector Machine a K Nearest Neighbors ke klasifikaci poruchy stroje. Porovnej, který algoritmus dosáhl lepších výsledků.

**Pozor, máme hodně dat, běží to dlouho:), na mém počítači asi 20 a 50 sekund.**

In [391]:
# Stejný postup jako v prvním úkolu

model_3 = KNeighborsClassifier()
params_3 = {"n_neighbors": [3, 5, 11, 15]}
# ...hodnoty n_neighbors zvoleny náhodně jen jako ukázka alternativy k range()

clf_3 = GridSearchCV(model_3, params_3, scoring="accuracy")
clf_3.fit(X2, y2)

print(clf_3.best_params_)
print(f"Best accuracy: {round(clf_3.best_score_, 2)}")

{'n_neighbors': 15}
Best accuracy: 0.97


In [392]:
# Stejný postup jako v prvním úkolu

model_4 = SVC()
params_4 = {"kernel": ["linear", "rbf"], "decision_function_shape": ["ovo", "ovr"]}

clf_4 = GridSearchCV(model_4, params_4, scoring="accuracy")
clf_4.fit(X2, y2)

print(clf_4.best_params_)
print(f"Best accuracy: {round(clf_4.best_score_, 2)}")

{'decision_function_shape': 'ovo', 'kernel': 'linear'}
Best accuracy: 1.0
